## Context Relevance Feedback Requirements
1. Relevance requires adherence to the entire query.
2. Long context with small relevant chunks are relevant.
3. Context that provides no answer can still be relevant.
4. Feedback mechanism should differentiate between seeming and actual relevance.
5. Relevant but inconclusive statements should get increasingly high scores as they are more helpful for answering the query.

Below are examples of usage. Find the [results](#test-results) of the tests tabulated at the bottom.

In [ ]:
# Import relevance feedback function
from trulens_eval.feedback import GroundTruthAgreement, OpenAI as fOpenAI
from trulens_eval import TruBasicApp, Feedback, Tru, Select
from test_cases import context_relevance_golden_set
fopenai = fOpenAI()

import openai

Tru().reset_database()

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "..."

In [ ]:
# Answer rlevance
def wrapped_relevance(input, output):
    return fopenai.relevance(input, output)

# Relevancw with chain of thought reasoning
def wrapped_relevance_with_cot(input, output):
    return fopenai.relevance_with_cot_reasons(input, output)

In [ ]:
# Create a Feedback object using the numeric_difference method of the ground_truth object
ground_truth = GroundTruthAgreement(context_relevance_golden_set)
# Call the numeric_difference method with app and record
f_groundtruth = Feedback(ground_truth.numeric_difference, name = "Relevance Smoke Test").on_input().on(Select.Record.calls[0].args.args[1]).on_output()

In [ ]:
tru_wrapped_relevance = TruBasicApp(wrapped_relevance, app_id = "context relevance", feedbacks=[f_groundtruth])
tru_wrapped_relevance_with_cot = TruBasicApp(wrapped_relevance_with_cot, app_id = "context relevance with cot reasoning", feedbacks=[f_groundtruth])

In [ ]:
for i in range(len(context_relevance_golden_set)):
    prompt = context_relevance_golden_set[i]["query"]
    response = context_relevance_golden_set[i]["response"]
    tru_wrapped_relevance.call_with_record(prompt, response)
    tru_wrapped_relevance_with_cot.call_with_record(prompt, response)

In [ ]:
Tru().run_dashboard()